In [17]:
# load basics library
import pandas as pd
import numpy as np
import calendar
from sklearn.cluster import MiniBatchKMeans
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.multiclass import type_of_target

In [18]:
def clean_data(df):
    df_ = df.copy()
    # remove the 2016-01-23 data since its too less comapre others days,
    # maybe quality is not good
    # df_ = df_[(df_.pickup_date != '2016-01-23') & (df_.dropoff_date != '2016-01-23')]
    # potential passenger_count outlier
    df_ = df_[(df_['passenger_count'] <= 6) & (df_['passenger_count'] > 0)]
    return df_

### ================================================ ###


def load_data():
    df_train = pd.read_csv('../data/train.csv')
    df_test = pd.read_csv('../data/test.csv')
    # sample train data for fast job
    # df_train = df_train.sample(n=100)
    # clean train data
    df_train_ = clean_data(df_train)
    df_test = clean_data(df_test)
    # merge train and test data for fast process and let model view test data when training as well
    df_all = pd.concat([df_train_, df_test], axis=0)
    return df_all, df_train_, df_test

In [19]:
df_all_, df_train, df_test = load_data()
print (df_all_.head())
# modeling
features = ['VendorID',
            'RatecodeID',
            'PULocationID',
            'DOLocationID',
            'passenger_count',
            'trip_distance',
            'fare_amount',
            'extra',
            'tolls_amount',
            'total_amount',
            'payment_type',
            'duration',
            'day_of_week',
            'speed',
            'tip_amount']
      
X_train = df_all_[features].values
y_train = X_train[:, X_train.shape[1]-1]

X_test = df_test[features].values
y_test = X_test[:, X_train.shape[1]-1]

y_train = np.ravel(y_train)
y_test = np.ravel(y_test)
print (df_all_[features].head())

   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2017-01-07 12:47:25   2017-01-07 13:07:19                  N   
1         2  2017-01-21 12:21:04   2017-01-21 12:22:57                  N   
2         2  2017-01-04 08:53:29   2017-01-04 09:08:15                  N   
3         2  2017-01-18 18:17:40   2017-01-18 18:24:12                  N   
4         2  2017-01-09 14:56:30   2017-01-09 15:17:13                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0           1            40           141              1.0           2.91   
1           1            74            74              1.0           0.36   
2           1            65           143              2.0           2.31   
3           1           165            40              1.0           0.96   
4           1            94           120              1.0           2.86   

   fare_amount ...   ehail_fee  improvement_surcharge  total_amount  \
0  

In [20]:
sample_size = 5000

X_train = X_train[:sample_size]
y_train = y_train[:sample_size]

X_test = X_test[:sample_size]
y_test = y_test[:sample_size]

In [21]:
# train xgb
xgb = XGBRegressor(n_estimators=1000, max_depth=12, min_child_weight=150,
                   subsample=0.7, colsample_bytree=0.3)
y_test = np.zeros(len(X_test))

In [23]:
for i, (train_ind, val_ind) in enumerate(KFold(n_splits=2, shuffle=True,
                                               random_state=1989).split(X_train)):
    print('----------------------')
    print('Training model #%d' % i)
    print('----------------------')

    xgb.fit(X_train[train_ind], y_train[train_ind],
            eval_set=[(X_train[val_ind], y_train[val_ind])],
            early_stopping_rounds=10, verbose=25)

    y_test += xgb.predict(X_test, ntree_limit=xgb.best_ntree_limit)
y_test /= 2

----------------------
Training model #0
----------------------
[0]	validation_0-rmse:1.91461
Will train until validation_0-rmse hasn't improved in 10 rounds.
[25]	validation_0-rmse:1.02815
[50]	validation_0-rmse:0.841787
[75]	validation_0-rmse:0.783748
[100]	validation_0-rmse:0.76696
[125]	validation_0-rmse:0.757514
Stopping. Best iteration:
[138]	validation_0-rmse:0.752371

----------------------
Training model #1
----------------------
[0]	validation_0-rmse:2.08856
Will train until validation_0-rmse hasn't improved in 10 rounds.
[25]	validation_0-rmse:1.31898
[50]	validation_0-rmse:1.1808
[75]	validation_0-rmse:1.13081
[100]	validation_0-rmse:1.11815
[125]	validation_0-rmse:1.10455
[150]	validation_0-rmse:1.10124
[175]	validation_0-rmse:1.09583
[200]	validation_0-rmse:1.0917
[225]	validation_0-rmse:1.08998
Stopping. Best iteration:
[234]	validation_0-rmse:1.0892



In [25]:
# need to transform into 1D array
type_of_target(y_train)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)

In [26]:
from xgboost import XGBClassifier

# CV model
model = XGBClassifier()
kfold = StratifiedKFold(n_splits=10, random_state=7)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/home/searoc/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/searoc/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/searoc/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/searoc/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

Accuracy: 95.74% (8.50%)


/home/searoc/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
